<a href="https://colab.research.google.com/github/Hamxea/Mutlti-label-Classification/blob/master/Multi-label%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Ini source lain kudapet, masik ku utak atik
jalanin nya di google colab soalnya pake Keras
trus ntar upload file scv ke colab nya, sama upload file glove.6B.100d (1).txt (download dari sini -> https://www.kaggle.com/danielwillgeorge/glove6b100dtxt/version/1)

masik ku edit edit jadi sabar y gays T_T
"""

In [ ]:
#!pip install keras 
#!pip install BeautifulSoup4
# dua ini jalanin kalo perlu aj ^^

#from google.colab import files
#uploaded = files.upload()

Testng CNN, RNN, LSTM

In [ ]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
os.environ['KERAS_BACKEND']='theano' # Why theano why not
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.switch_backend('agg')
%matplotlib inline

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
train_df = pd.read_csv("train.csv").fillna("sterby")
test_df = pd.read_csv("test.csv").fillna("sterby")

In [ ]:
train_df.count()

In [ ]:
train_df.head(8)

In [ ]:
"""
max_features = 20000  # number of words we want to keep
maxlen = 100  # max length of the comments in the model
batch_size = 64  # batch size for the model
embedding_dims = 20  # dimension of the hidden variable, i.e. the embedding dimension
"""

In [ ]:
X_train = train_df["comment_text"].values
y_train = train_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test_df["comment_text"].values

In [ ]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
x_train = tokenizer.texts_to_sequences(X_train) #Recheck here
x_test = tokenizer.texts_to_sequences(X_test)

word_index = tokenizer.word_index
print('Number of train squences',len(x_train))
print('Number of test sequences',len(x_test))

In [ ]:
x_train = pad_sequences(x_train, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(x_test, maxlen=MAX_SEQUENCE_LENGTH)

#labels = to_categorical(np.asarray(labels))
print('Shape of Data Tensor:', x_train.shape)
print('Shape of Label Tensor:', x_test.shape)


indices = np.arange(x_train.shape[0])
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]
nb_validation_samples = int(VALIDATION_SPLIT * x_train.shape[0])

x_train = x_train[:-nb_validation_samples]
y_train = x_test[:-nb_validation_samples]
x_val = x_train[-nb_validation_samples:]
y_val = x_test[-nb_validation_samples:]

In [ ]:
"""
acronum=sorted(set(y_train))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

y_train = y_train.apply(fun)

"""

In [ ]:
embeddings_index = {}
f = open('glove.6B.100d (1).txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

In [ ]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,trainable=True)

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
#preds = Dense(len(y_train), activation='softmax')(l_dense)

model = Model(sequence_input, l_dense)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("Simplified convolutional neural network")
model.summary()
cp=ModelCheckpoint('model_cnn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)

In [ ]:
#history=model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=15, batch_size=2,callbacks=[cp])